# Titanic Survival Prediction System
## Model Development
**Name:** Onafowokan Testament

**Matric:** 21cg029905

**Algorithm:** Logistic Regression

**Features Used:** Pclass, Sex, Age, Fare, Embarked

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
import warnings
warnings.filterwarnings('ignore')

print('All libraries imported successfully!')

## 2. Load the Titanic Dataset

In [ ]:
# Load the Titanic dataset
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

print('Dataset shape:', df.shape)
print('\nFirst few rows:')
print(df.head())
print('\nDataset Info:')
print(df.info())
print('\nMissing Values:')
print(df.isnull().sum())

## 3. Data Preprocessing

In [ ]:
# Select features and target variable
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
X = df[features].copy()
y = df['Survived'].copy()

print('Selected Features:', features)
print('Target Variable: Survived')
print('\nDataset shape - X:', X.shape, 'y:', y.shape)

### 3a. Handle Missing Values

In [ ]:
# Check for missing values
print('Missing values before handling:')
print(X.isnull().sum())

# Handle missing values
# Age: fill with median
X['Age'].fillna(X['Age'].median(), inplace=True)

# Embarked: fill with mode
X['Embarked'].fillna(X['Embarked'].mode()[0], inplace=True)

# Fare: fill with median
X['Fare'].fillna(X['Fare'].median(), inplace=True)

print('\nMissing values after handling:')
print(X.isnull().sum())

### 3b & 3c. Encode Categorical Variables

In [ ]:
# Create a copy for encoding
X_encoded = X.copy()

# Encode Sex (Male=1, Female=0)
X_encoded['Sex'] = (X_encoded['Sex'] == 'male').astype(int)

# Encode Embarked using one-hot encoding
X_encoded = pd.get_dummies(X_encoded, columns=['Embarked'], drop_first=True)

print('Features after encoding:')
print(X_encoded.head())
print('\nFeature columns:', X_encoded.columns.tolist())

### 3d. Feature Scaling

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the features
X_scaled = scaler.fit_transform(X_encoded)

# Convert back to DataFrame to maintain column names
X_scaled = pd.DataFrame(X_scaled, columns=X_encoded.columns)

print('Features after scaling:')
print(X_scaled.head())
print('\nScaling statistics:')
print('Mean of scaled features:', X_scaled.mean().round(4).values)
print('Std of scaled features:', X_scaled.std().round(4).values)

## 4. Train-Test Split

In [ ]:
# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print('Training set size:', X_train.shape)
print('Testing set size:', X_test.shape)
print('\nClass distribution in training set:')
print(y_train.value_counts())

## 5. Train Logistic Regression Model

In [ ]:
# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

print('Logistic Regression model trained successfully!')
print('Model coefficients:', model.coef_)
print('Model intercept:', model.intercept_)

## 6. Model Evaluation

In [ ]:
# Make predictions on training and test sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculate accuracy scores
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print('='*60)
print('MODEL PERFORMANCE')
print('='*60)
print(f'Training Accuracy: {train_accuracy:.4f}')
print(f'Testing Accuracy: {test_accuracy:.4f}')
print('\n' + '='*60)
print('CLASSIFICATION REPORT (Test Set)')
print('='*60)
print(classification_report(y_test, y_test_pred, target_names=['Did Not Survive', 'Survived']))

print('\n' + '='*60)
print('CONFUSION MATRIX (Test Set)')
print('='*60)
print(confusion_matrix(y_test, y_test_pred))

## 7. Save the Model

In [ ]:
# Save the trained model using joblib
joblib.dump(model, 'titanic_survival_model.pkl')
print('✓ Model saved as: titanic_survival_model.pkl')

# Save the scaler for preprocessing new data
joblib.dump(scaler, 'scaler.pkl')
print('✓ Scaler saved as: scaler.pkl')

## 8. Load and Test the Saved Model

In [ ]:
# Load the saved model
loaded_model = joblib.load('titanic_survival_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')

print('✓ Model loaded successfully!')
print('✓ Scaler loaded successfully!')

# Make predictions with the loaded model
y_pred_loaded = loaded_model.predict(X_test)
loaded_model_accuracy = accuracy_score(y_test, y_pred_loaded)

print(f'\nLoaded Model Test Accuracy: {loaded_model_accuracy:.4f}')
print(f'Original Model Test Accuracy: {test_accuracy:.4f}')
print(f'Match: {loaded_model_accuracy == test_accuracy}')

## 9. Example Prediction

In [ ]:
# Example: Make a prediction with a new passenger
# Features: Pclass, Sex, Age, Fare, Embarked
new_passenger = pd.DataFrame({
    'Pclass': [3],
    'Sex': ['male'],
    'Age': [25],
    'Fare': [7.75],
    'Embarked': ['S']
})

# Apply the same preprocessing
new_passenger_encoded = new_passenger.copy()
new_passenger_encoded['Sex'] = (new_passenger_encoded['Sex'] == 'male').astype(int)
new_passenger_encoded = pd.get_dummies(new_passenger_encoded, columns=['Embarked'], drop_first=True)

# Ensure all columns match
for col in X_encoded.columns:
    if col not in new_passenger_encoded.columns:
        new_passenger_encoded[col] = 0

new_passenger_encoded = new_passenger_encoded[X_encoded.columns]

# Scale the features
new_passenger_scaled = loaded_scaler.transform(new_passenger_encoded)

# Make prediction
prediction = loaded_model.predict(new_passenger_scaled)[0]
probability = loaded_model.predict_proba(new_passenger_scaled)[0]

print('='*60)
print('EXAMPLE PREDICTION')
print('='*60)
print('Passenger Details:')
print(f'  Pclass: {new_passenger["Pclass"].values[0]}')
print(f'  Sex: {new_passenger["Sex"].values[0]}')
print(f'  Age: {new_passenger["Age"].values[0]}')
print(f'  Fare: {new_passenger["Fare"].values[0]}')
print(f'  Embarked: {new_passenger["Embarked"].values[0]}')
print('\nPrediction Result:')
result = 'SURVIVED' if prediction == 1 else 'DID NOT SURVIVE'
print(f'  Predicted: {result}')
print(f'  Confidence (Did Not Survive): {probability[0]:.4f}')
print(f'  Confidence (Survived): {probability[1]:.4f}')